In [64]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

rs = 3791

# Load & Prepare Data

In [28]:
df = pd.read_csv("../data/daily_merged_hydro_climate.csv")  

#ensure that date is an date_time object
df['date'] = pd.to_datetime(df['date'])

#Renam Columns
df = df.rename(columns={
    "Temperature (°C)_mean": "temp_mean",
    "Temperature (°C)_min": "temp_min",
    "Temperature (°C)_max": "temp_max",
    "Dewpoint Temperature (°C)_mean": "dew_point_mean",
    "Dewpoint Temperature (°C)_min": "dew_point_min",
    "Dewpoint Temperature (°C)_max": "dew_point_max",
    "U Wind Component (m/s)_mean": "u",
    "V Wind Component (m/s)_mean": "v",
    "Total Precipitation (mm)_sum": "tot_precip",
    "Snowfall (mm)_sum": "snowfall",
    "Snow Cover (%)_mean": "snow_cover"
    })


The data is missing some information for certain days within the time series. To prepare the data for modeling, we **reindex the dataset so that every Source has an entry for every Date** between the earliest and latest timestamp in the dataset. This ensures that the time series is complete and consistent across all sources.

In [29]:
# Get the complete date range in the dataset
all_dates = pd.date_range(df["date"].min(), df["date"].max())

# Get all unique values for "Source" 
all_sources = df["Source"].unique()

#Build a MultiIndex with all combinations of Sources × Dates
idx = pd.MultiIndex.from_product([all_sources, all_dates], names=["Source","date"])

#Reindex the dataframe (row for every Source-Date combiation)
df_full = df.set_index(["Source","date"]).reindex(idx).reset_index()

#Fill missing kWh values with 0 
df_full["kwh_sum"] = df_full["kwh_sum"].fillna(0)

#Identify all climate-related columns and forward values so every Source has the same climate data on a given Dat
climate_cols = [c for c in df.columns if c not in ["Source","kwh_sum","date"]]
for c in climate_cols:
    df_full[c] = df_full.groupby("date")[c].transform("first")

### Feature Engeneering

To help the model capture **temporal dynamics** in the data, we generate lagged and rolling features for both energy consumption and climate variables.

In [30]:
lags = [1, 2, 3]  # example: previous day, 2 days ago, 3 days ago, 1 week ago, 1 month agor   
for lag in lags:
    df[f'kwh_sum{lag}'] = df['kwh_sum'].shift(lag).fillna(0).astype(int)  #fill NaNs, generated durinng lag feature creatiion with 0

lags = [7, 30]  # example: previous day, 2 days ago, 3 days ago, 1 week ago, 1 month agor   
for lag in lags:
    df[f'kwh_sum{lag}'] = df['kwh_sum'].rolling(window=30, min_periods=1).mean()   #fill NaNs, generated durinng lag feature creatiion with 0

# moving average lags for relevant climate features
df['temp_mean_30d'] = df['temp_mean'].rolling(window=30, min_periods=1).mean()    #long memory
df['temp_mean_30d'] = df['temp_mean_30d'].fillna(0)

df['dew_point_mean_30d'] = df['dew_point_mean'].rolling(window=7, min_periods=1).mean()    #long memory
df['dew_point_mean_30d'] = df['dew_point_mean_30d'].fillna(0)

df['tot_precip_30d'] = df['tot_precip'].rolling(window=30, min_periods=1).mean()    #long memory
df['tot_precip_30d'] = df['tot_precip_30d'].fillna(0)

df['snowfall_30d'] = df['snowfall'].rolling(window=30, min_periods=1).mean()    #long memory
df['snowfall_30d'] = df['snowfall_30d'].fillna(0)

df['snow_cover_30d'] = df['snow_cover'].rolling(window=30, min_periods=1).mean()    #long memory
df['snow_cover_30d'] = df['snow_cover_30d'].fillna(0)        

### Train Model

In [31]:
from xgboost import XGBRegressor
from timeit import default_timer as timer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error   
from sklearn.preprocessing import LabelEncoder

In [32]:
X = df[["temp_mean",
    "dew_point_mean",
    "u",
    "v",
    "tot_precip",
    "snowfall",
    "snow_cover",
    'kwh_sum1',
    'kwh_sum2',
    'kwh_sum3',
    'kwh_sum7',
    'kwh_sum30',
    'temp_mean_30d',
    'dew_point_mean_30d',
    'tot_precip_30d',
    'snowfall_30d',
    'snow_cover_30d',
    'consumer_device']]   
y = df["kwh_sum"]

In [50]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=rs, shuffle=True, stratify=X["consumer_device"])

#drop consumer_device

X_train.drop('consumer_device', axis = 1, inplace = True)
X_test.drop('consumer_device', axis = 1, inplace = True)

#safe features used for model training 
feature_cols = X_train.columns

In [34]:
# create model instance
bst = XGBRegressor()     

bst.fit(X_train, y_train)

,objective,'reg:squarederror'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [35]:
# Make predictions on test set 
preds_test = bst.predict(X_test)
print(preds_test)

# Make predictions on train set set 
preds_train = bst.predict(X_train)
print(preds_train)

[ 1.2438337e+00 -1.0674202e-03  2.9133866e-03 ...  2.9133866e-03
  2.9133866e-03  2.9133866e-03]
[ 2.9133866e-03  2.9133866e-03 -1.0674202e-03 ...  7.8692716e-01
  1.1957928e+00  2.9133866e-03]


In [36]:
# Evaluate the model on train data
r2_train = r2_score(y_train, preds_train)
rmse_train = np.sqrt(mean_squared_error(y_train, preds_train))

print('r squared on training data is: ', round(r2_train,3))
print("RMSE on training data is: ", round(rmse_train,3))

# Evaluate the model on test data
r2 = r2_score(y_test, preds_test)
rmse = np.sqrt(mean_squared_error(y_test, preds_test))

print('r squared for test data is:', round(r2,3))
print("RMSE for test data is: ", round(rmse,3))



r squared on training data is:  0.391
RMSE on training data is:  2.893
r squared for test data is: 0.042
RMSE for test data is:  3.815


The target variable (`kwh_sum`) contains many **zero values**, including long periods of several weeks with no energy availability. Consequently:   

*  **R² becomes misleading**  
* **RMSE remains meaningful** 

As the dataset is dominated by zeros, R² may not be a fair indicator of model quality. RMSE (or other absolute error measures) in this case provides a more reliable view of predictive performance.

### Hyperparameter Tuning 

We first start with a broader search to identify the most influential parameters and their approximate ranges, reducing the search space for the next stage. The target is to optimize the RMSE.

In [38]:
# Sefine Paarameters for tuning
param_dist_stage1 = {
    "n_estimators": [100, 500, 1000],
    "max_depth": [3, 6, 9],       #Shallow trees generalize better, deep trees tend to overfit
    "learning_rate": [0.01, 0.05, 0.1],
    "subsample": [0.6, 0.8, 1.0],   # Adds randomness, prevents overfitting
    "colsample_bytree": [0.5, 0.7, 1.0],
}

In [39]:
grid = GridSearchCV(estimator=bst,
                    param_grid=param_dist_stage1,
                    cv=4,
                    scoring='neg_root_mean_squared_error',  # or 'r2' 
                    verbose=1,
                    n_jobs=-1)

In [40]:
# Fit object to data
start = timer()
grid.fit(X_train, y_train)
end = timer()
gs_time = end-start

Fitting 4 folds for each of 243 candidates, totalling 972 fits


KeyboardInterrupt: 

In [ ]:
# Best score
print('Best score:', round(grid.best_score_, 3))

# Best parameters
best_params_stage1  = grid.best_params_
print('Best parameters:', best_params_stage1)


AttributeError: 'GridSearchCV' object has no attribute 'best_score_'

The hyperparameter tuning was run with the goal to optimize `scoring='neg_root_mean_squared_error'`. In scikit-learn, all scoring functions are defined so that **higher values mean better performance**. Since RMSE is an error metric (where *lower is better*), scikit-learn simply returns the **negative RMSE** to stay consistent with this convention. Hence, for the actual best RMSSE after tuning, just remove the negative sign.

The tuned model achieved a cross-validated RMSE of **3.499**, which is an improvement over both the baseline (~3.8) and the benchmark (~4.5).

The best combination of parameters is:
* 'colsample_bytree' = 0.7
* 'learning_rate'= 0.01
* 'max_depth'= 3
* 'n_estimators'= 500
* 'subsample'= 1.0


### Further Fine-Tuning

A second, fine-tuned grid search focused on less influential parameters while keeping the previously optimized parameters fixed.

In [ ]:
# Sefine Paarameters for tuning
param_dist_stage2  = {   
    "gamma": [0, 0.1, 0.5],
    "reg_alpha": [0, 0.1, 1],
    "reg_lambda": [1, 2, 5]
}

# Merge: keep best values for stage 1, vary the new ones
param_dist_fTune = {**best_params_stage1, **param_dist_stage2}

In [ ]:
"""
param_dist_fTune = { "n_estimators": [500],
                    "max_depth": [3],
                    "learning_rate": [0.01],
                    "subsample": [1.0],
                    "colsample_bytree": [0.7],
                    "gamma": [0, 0.1, 0.5],
                    "reg_alpha": [0, 0.1, 1],
                    "reg_lambda": [1, 2, 5] }
"""

In [43]:
grid = GridSearchCV(estimator=bst,
                    param_grid=param_dist_fTune,
                    cv=4,
                    scoring='neg_root_mean_squared_error',  # or 'r2' 
                    verbose=1,
                    n_jobs=-1)

In [44]:
# Fit object to data
start = timer()
grid.fit(X_train, y_train)
end = timer()
gs_time = end-start

Fitting 4 folds for each of 27 candidates, totalling 108 fits


In [45]:
# Best score
print('Best score:', round(grid.best_score_, 3))

# Best parameters
best_params_stage2  = grid.best_params_
print('Best parameters:', best_params_stage2)

Best score: -3.498
Best parameters: {'colsample_bytree': 0.7, 'gamma': 0.5, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 500, 'reg_alpha': 0, 'reg_lambda': 5, 'subsample': 1.0}


The small change indicates that the model was already close to optimal with the first-stage parameters, and further tuning of regularization had only minimal impact.

### Train Optimized Model

In [46]:
# create model instance using the best parameters from tuning
bst = XGBRegressor(**grid.best_params_)

# Train on your full training data
bst.fit(X_train, y_train)

,objective,'reg:squarederror'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,0.7
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [47]:
# Make predictions on test set 
preds_test = bst.predict(X_test)
print(preds_test)

# Make predictions on train set set 
preds_train = bst.predict(X_train)
print(preds_train)

[1.0164332  0.01438829 0.01611627 ... 0.03248273 0.02060819 0.01438829]
[0.01611627 0.01611627 0.02060819 ... 0.46091184 1.9807371  0.01913098]


In [48]:
# Evaluate the model on train data
r2_train = r2_score(y_train, preds_train)
rmse_train = np.sqrt(mean_squared_error(y_train, preds_train))

print('r squared on training data is: ', round(r2_train,3))
print("RMSE on training data is: ", round(rmse_train,3))

# Evaluate the model on test data
r2 = r2_score(y_test, preds_test)
rmse = np.sqrt(mean_squared_error(y_test, preds_test))

print('r squared for test data is:', round(r2,3))
print("RMSE for test data is: ", round(rmse,3))

r squared on training data is:  0.124
RMSE on training data is:  3.47
r squared for test data is: 0.113
RMSE for test data is:  3.669


# Prepare Submission File for Zindi

In thiss section, the final model is used to generate sequential predictions for each source over the validation period. Lag and rolling features weraree dynamically updated after each predicted day to simulate real-world forecasting. The predictions were merged with the sample submission template to produce a properly formatted CSV file ready for upload to Zindi.

In [51]:
#  Load the climate data

file_path = "../data/Kalam Climate Data.xlsx"   
climate_df = pd.read_excel(file_path)

climate_df['Date Time'] = pd.to_datetime(climate_df['Date Time'])

# Extract date

climate_df['date'] = climate_df['Date Time'].dt.date

# Daily aggregation
daily_climate = climate_df.groupby('date').agg({
    'Temperature (°C)': ['mean', 'min', 'max'],
    'Dewpoint Temperature (°C)': ['mean', 'min', 'max'],
    'U Wind Component (m/s)': 'mean',
    'V Wind Component (m/s)': 'mean',
    'Total Precipitation (mm)': 'sum',
    'Snowfall (mm)': 'sum',
    'Snow Cover (%)': 'mean'
})

# Flatten columns
daily_climate.columns = ['_'.join(col).strip() for col in daily_climate.columns.values]

# Reset index
daily_climate = daily_climate.reset_index()

# Define start and end dates
start_date = pd.to_datetime('2024-09-24')
end_date   = pd.to_datetime('2024-10-24')

daily_climate["date"] = pd.to_datetime(daily_climate["date"])

# Filter DataFrame
extra_month_df = daily_climate[(daily_climate['date'] >= start_date) & (daily_climate['date'] <= end_date)]

In [ ]:
#rename colums to match your feature_cols
extra_month_df = extra_month_df.rename(columns={
    'Temperature (°C)_mean': 'temp_mean',
    'Dewpoint Temperature (°C)_mean': 'dew_point_mean',
    'U Wind Component (m/s)_mean': 'u',
    'V Wind Component (m/s)_mean': 'v',
    'Total Precipitation (mm)_sum': 'tot_precip',
    'Snowfall (mm)_sum': 'snowfall',
    'Snow Cover (%)_mean': 'snow_cover'
})

In [59]:
preds = []

for src in df["Source"].unique():
    hist = df[df["Source"]==src].copy()
    extra_src = extra_month_df.copy()
    extra_src["Source"] = src
    
    lag_hist = hist.tail(30).reset_index(drop=True)
    rows = []
    
    for _, row in extra_src.iterrows():
        r = row.copy()
        
        # Lag features for target
        for lag in [1,2,7,14]:
            r[f"kwh_sum{lag}"] = lag_hist["kwh_sum"].iloc[-lag] if len(lag_hist) >= lag else 0
        
        # Rolling mean features for target
        for w in [3,7,14]:
            r[f"roll_mean_{w}"] = lag_hist["kwh_sum"].iloc[-w:].mean() if len(lag_hist) >= w else lag_hist["kwh_sum"].mean()
        
        # Add rolling averages for climate features (30d) if needed
        for feat in ["temp_mean","dew_point_mean","tot_precip","snowfall","snow_cover"]:
            r[f"{feat}_30d"] = lag_hist[feat].iloc[-30:].mean() if len(lag_hist) >= 30 else lag_hist[feat].mean()
        
        # Ensure all features exist
        X_ex = pd.DataFrame([r])
        for col in feature_cols:
            if col not in X_ex.columns:
                X_ex[col] = 0
        X_ex = X_ex[feature_cols]
        
        # Predict
        r["pred_kwh"] = max(0, bst.predict(X_ex)[0])
        
        # Update history
        lag_hist = pd.concat([lag_hist, pd.DataFrame([{"kwh_sum": r["pred_kwh"], **{c: r[c] for c in ["temp_mean","dew_point_mean","u","v","tot_precip","snowfall","snow_cover"]}}])], ignore_index=True)
        
        rows.append(r)
    
    preds.append(pd.DataFrame(rows))

preds_df = pd.concat(preds).reset_index(drop=True)

In [ ]:
preds_df = preds_df.rename(columns={"date": "Date"}) #Done to follow the conventions of the Zindii template

In [63]:
sub = pd.read_csv("../data/SampleSubmission.csv")
sub["Date"] = sub["ID"].apply(lambda x: x.split("_")[0])
sub["Source"] = sub["ID"].apply(lambda x: "_".join(x.split("_")[1:]))
sub["Date"] = pd.to_datetime(sub["Date"])

preds_out = preds_df.copy()
preds_out["Date"] = pd.to_datetime(preds_out["Date"])

submission = sub.merge(preds_out[["Date","Source","pred_kwh"]], on=["Date","Source"], how="left")
submission = submission[["ID","pred_kwh"]]
submission.to_csv("MySubmission_XGBoost_V1.csv", index=False)

### FAZIT – Overall Model Performance

When testing the model on new, unnknown climate data, itt achieved a Zindi privat score of 7.618465289, which would have ranked it on position 321. The current model is functional and can generate submission files, but its predictive accuracy is limited. Further pursuit of this particular modeling approach is unlikely to yield substantial gains.